In [ ]:
import dendron
from dendron.actions.causal_lm_action import CausalLMActionConfig, CausalLMAction

In [ ]:
chat_behavior_cfg = CausalLMActionConfig(load_in_4bit=True,
                                         max_new_tokens=128,
                                         do_sample=True,
                                         top_p=0.95,
                                         use_flash_attn_2=True,
                                         model_name='openchat/openchat-3.5-0106')

In [ ]:
chat_node = CausalLMAction('chat_node', chat_behavior_cfg)
tree = dendron.BehaviorTree("chat_tree", chat_node)

In [ ]:
chat = [
    {"role": "GPT4 Correct User", "content": "Hello, how are you?"},
    {"role" : "GPT4 Correct Assistant", "content" : "I am well. How are you?"}
]

In [ ]:
def chat_to_str(self, chat):
    return self.tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

tree.root.set_input_processor(chat_to_str)

In [ ]:
def str_to_chat(self, str):
    key = "GPT4 Correct Assistant:"
    idx = str.rfind(key)
    response = str[idx+len(key):]
    chat = self.blackboard[self.input_key]
    chat.append({"role" : "GPT4 Correct Assistant", "content" : response})
    return chat

tree.root.set_output_processor(str_to_chat)

In [ ]:
while True:
    input_str = input("Input: ")
    chat.append({"role": "GPT4 Correct User", "content" : input_str})
    tree.blackboard["in"] = chat

    tree.tick_once()

    print("Output: ", tree.blackboard["out"][-1]["content"])
    if "Goodbye" in tree.blackboard["out"][-1]["content"]:
        break

In [ ]:
print(tree.blackboard["out"])

In [ ]:
print(tree.blackboard)